In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/미세먼지_PM2.5__월별_도시별_대기오염도_20231116153601.csv', encoding='ansi')

In [3]:
df.head(3)

,구분(1),구분(2),2015.01,2015.02,2015.03,2015.04,2015.05,2015.06,2015.07,2015.08,...,2022.05,2022.06,2022.07,2022.08,2022.09,2022.10,2022.11,2022.12,2023.01,2023.02
0,총계,소계,28,32,30,24,26,26,21,23,...,17,12,13,10,11,14,23,20,24,28
1,서울특별시,서울특별시,25,30,30,21,22,20,19,22,...,17,12,16,11,11,14,22,19,26,30
2,부산광역시,부산광역시,27,31,27,24,31,30,21,23,...,17,10,12,10,10,11,19,18,21,22


In [4]:
long_df = df.melt(id_vars=['구분(1)', '구분(2)'], 
                  var_name='STD_YYYYMM', 
                  value_name='PM2.5')

In [5]:
long_df.head(3)

,구분(1),구분(2),STD_YYYYMM,PM2.5
0,총계,소계,2015.01,28
1,서울특별시,서울특별시,2015.01,25
2,부산광역시,부산광역시,2015.01,27


In [6]:
long_df.to_csv('data/test/pm25.csv', index=False)

In [6]:
long_df.rename(columns={'구분(1)': 'SIDO'}, inplace=True)

In [7]:
long_df.head()

,SIDO,구분(2),STD_YYYYMM,PM2.5
0,총계,소계,2015.01,28
1,서울특별시,서울특별시,2015.01,25
2,부산광역시,부산광역시,2015.01,27
3,대구광역시,대구광역시,2015.01,33
4,인천광역시,인천광역시,2015.01,31


In [8]:
long_df['PM2.5'].dtype

dtype('O')

In [9]:
long_df['PM2.5'] = pd.to_numeric(long_df['PM2.5'], errors='coerce') 

# 오브젝트 타입에서의 변환

In [11]:
# SIDO, STD_YYYYMM 열을 기준으로 그룹바이 한후 PM10의 평균값을 새로운 콜롬에 저장
long_df['AVG_PM2.5'] = long_df.groupby(['SIDO', 'STD_YYYYMM'])['PM2.5'].transform('mean')

In [12]:
# 평균값을 소수점 둘째자리까지 반올림
long_df['AVG_PM2.5'] = long_df['AVG_PM2.5'].round(2)

In [13]:
df2 = long_df.drop_duplicates(subset=['SIDO', 'STD_YYYYMM', 'AVG_PM2.5'])
# SIDO와 STD_YYYYMM 열을 기준으로 중복을 제거
# 중복된 데이터를 제거하면 데이터가 정렬됨 

In [20]:
df2.to_csv('data/test/pm25.csv', index=False)

In [17]:
df2.head(3)

,SIDO,STD_YYYYMM,AVG_PM2.5
0,총계,2015.01,28.0
1,서울특별시,2015.01,25.0
2,부산광역시,2015.01,27.0


In [16]:
# PM10 , 항목 , 단위 콜롬 제거
df2 = df2.drop(columns=['PM2.5', '구분(2)'])

In [19]:
# SIDO 열에 총계 또는 도평균을 포함하는 열을 필터링 
df2 = df2[~df2['SIDO'].str.contains('총계')]

In [22]:
# 'STD_YYYYMM' 열의 숫자가 아닌 문자를 제거
df2['STD_YYYYMM'] = df2['STD_YYYYMM'].str.replace('[^\d]', '', regex=True)

In [23]:
# 결과를 정수 타입으로 변환
df2['STD_YYYYMM'] = df2['STD_YYYYMM'].astype(int)


In [24]:
df2['STD_YYYYMM'].dtype

dtype('int32')

In [25]:
df2.to_csv('data/test/pm25_2.csv', index=False)

In [10]:
df = pd.read_csv('data/test/pm25_2.csv')

In [11]:
df.tail()

,SIDO,STD_YYYYMM,AVG_PM2.5,PM2_BAD
1661,전라북도,202302,30.93,N
1662,전라남도,202302,20.57,N
1663,경상북도,202302,24.04,N
1664,경상남도,202302,21.05,N
1665,제주특별자치도,202302,15.00,N


In [17]:
import numpy as np


초미세먼지(PM2.5)에 대한 대기환경 기준은 다음과 같습니다:

연간평균치 15㎍/㎥ 이하
24시간평균치 35㎍/㎥ 이하

In [14]:
# 'AVG_PM2.5'의 값이 35 이하이거나 NaN이면 'N', 35 초과면 'Y'를 'PM2_BAD' 열에 할당
df['PM2_35BAD'] = np.where((df['AVG_PM2.5'] <= 35) | pd.isna(df['AVG_PM2.5']), 'N', 'Y')


In [15]:
df.head()

,SIDO,STD_YYYYMM,AVG_PM2.5,PM2_BAD,PM2_35BAD
0,서울특별시,201501,25.0,N,N
1,부산광역시,201501,27.0,N,N
2,대구광역시,201501,33.0,N,N
3,인천광역시,201501,31.0,N,N
4,광주광역시,201501,27.0,N,N


In [16]:
df['PM2_35BAD'].value_counts()

PM2_35BAD
N    1601
Y      65
Name: count, dtype: int64

In [18]:
# AVG_납의 평균 계산
avg_pm2 = df['AVG_PM2.5'].mean()

# 조건에 따라 납_BAD에 Y 또는 N 할당
df['PM2_BAD'] = df['AVG_PM2.5'].apply(lambda x: 'Y' if x > avg_pm2 else 'N')

In [19]:
avg_pm2

21.689753694581277

In [20]:
df['PM2_BAD'].value_counts()

PM2_BAD
N    882
Y    784
Name: count, dtype: int64

In [21]:
df.to_csv('data/test/pm25_2.csv', index=False)